In [1]:
import pandas as pd
from Bio.SeqIO import parse
from sklearn.model_selection import StratifiedGroupKFold
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)

%load_ext autoreload
%autoreload 2

# Get SwissProt sequences

In [3]:
data = parse('uniprot-9606_proteome_human_reviewed_canonical_isoforms_191008.fasta', 'fasta')

seqs = []
for rec in data:
    seqs.append({'seq': str(rec.seq), 'id': rec.id, 'description': rec.description})

seqs = pd.DataFrame(seqs)
seqs['protein_ac'] = seqs['id'].apply(lambda x: x.split('|')[1])
seqs = seqs[~seqs['protein_ac'].str.contains('-')].copy()  # only retain canonical sequences
seqs['gene'] = seqs['description'].apply(lambda x: x.split('GN=')[1].split(' PE=')[0] if 'GN=' in x else None)
seqs

,seq,id,description,protein_ac,gene
0,MPQLSLSWLGLGPVAASPWLLLLLVGGSWLLARVLAWTYTFYDNCR...,sp|Q9HBI6|CP4FB_HUMAN,sp|Q9HBI6|CP4FB_HUMAN Cytochrome P450 4F11 OS=...,Q9HBI6,CYP4F11
1,MQRLMMLLATSGACLGLLAVAAVAAAGANPAQRDTHSLLPTHRRQK...,sp|P33151|CADH5_HUMAN,sp|P33151|CADH5_HUMAN Cadherin-5 OS=Homo sapie...,P33151,CDH5
3,MLQIGEDVDYLLIPREVRLAGGVWRVISKPATKEAEFRERLTQFLE...,sp|Q9H0W5|CCDC8_HUMAN,sp|Q9H0W5|CCDC8_HUMAN Coiled-coil domain-conta...,Q9H0W5,CCDC8
4,MSLPPEKASELKQLIHQQLSKMDVHGRIREILAETIREELAPDQQH...,sp|Q8TAP6|CEP76_HUMAN,sp|Q8TAP6|CEP76_HUMAN Centrosomal protein of 7...,Q8TAP6,CEP76
7,MARAGPRLVLSEEAVRAKSGLGPHRDLAELQSLSIPGTYQEKITHL...,sp|Q9P209|CEP72_HUMAN,sp|Q9P209|CEP72_HUMAN Centrosomal protein of 7...,Q9P209,CEP72
...,...,...,...,...,...
42412,MALIRKTFYFLFAMFFILVQLPSGCQAGLDFSQPFPSGEFAVCESC...,sp|Q8NG35|D105A_HUMAN,sp|Q8NG35|D105A_HUMAN Beta-defensin 105 OS=Hom...,Q8NG35,DEFB105A
42413,MRSMKALQKALSRAGSHCGRGGWGHPSRSPLLGGGVRHHLSEAAAQ...,sp|Q9UI32|GLSL_HUMAN,sp|Q9UI32|GLSL_HUMAN Glutaminase liver isoform...,Q9UI32,GLS2
42415,MNLPRAERLRSTPQRSLRDSDGEDGKIDVLGEEEDEDEEEAASQQF...,sp|Q12950|FOXD4_HUMAN,sp|Q12950|FOXD4_HUMAN Forkhead box protein D4 ...,Q12950,FOXD4
42416,MCSLPRGFEPQAPEDLAQRSLVELREMLKRQERLLRNEKFICKLPD...,sp|P0CAP2|GRL1A_HUMAN,sp|P0CAP2|GRL1A_HUMAN DNA-directed RNA polymer...,P0CAP2,POLR2M


# Labels

Please download Supplementary Table 3 '41564_2021_958_MOESM3_ESM.xlsx' from https://www.nature.com/articles/s41564-021-00958-0

Or direct download link: https://static-content.springer.com/esm/art%3A10.1038%2Fs41564-021-00958-0/MediaObjects/41564_2021_958_MOESM3_ESM.xlsx

In [4]:
labels = pd.read_excel('41564_2021_958_MOESM3_ESM.xlsx',  header=3, usecols="F:AT")
labels = labels.drop_duplicates(subset=['Reference', 'Assay type', 'Gene name', 'Uniprot accession'])
counts = dict(labels.groupby(['Gene name']).apply(lambda x: x['Reference'].unique().shape[0]))  # Counts
valid = labels.groupby(['Gene name']).count()['Functionally validated by authors']  # Functionally validated
labels = pd.DataFrame([counts, valid]).transpose().reset_index().rename(columns={0: 'n_publications', 1: 'n_functionally_validated', 'index': 'Gene name'})

In [5]:
labels['label'] = ((labels['n_publications'] >= 3) | (labels['n_functionally_validated'] >= 1.0)).astype(float)
labels.loc[(labels['n_publications'] < 3) & (labels['n_functionally_validated'] < 1.0), 'label'] = 'candidate'

# Merge Sequences with Labels

In [6]:
seqs_and_labels = pd.merge(seqs, labels[['Gene name', 'label']], 'left', left_on='gene', right_on='Gene name')
seqs_and_labels.loc[seqs_and_labels['label'].isna(), 'label'] = 0.0
seqs_and_labels = seqs_and_labels.drop(columns='Gene name')
seqs_and_labels.shape

(20415, 6)

In [7]:
# NaN indicates negatives, since these are ones that were not mentioned in the list of genes from Baggan et. al
seqs_and_labels['label'].value_counts(dropna=False)

0.0          15434
candidate     3936
1.0           1045
Name: label, dtype: int64

In [8]:
seqs_and_labels

,seq,id,description,protein_ac,gene,label
0,MPQLSLSWLGLGPVAASPWLLLLLVGGSWLLARVLAWTYTFYDNCR...,sp|Q9HBI6|CP4FB_HUMAN,sp|Q9HBI6|CP4FB_HUMAN Cytochrome P450 4F11 OS=...,Q9HBI6,CYP4F11,candidate
1,MQRLMMLLATSGACLGLLAVAAVAAAGANPAQRDTHSLLPTHRRQK...,sp|P33151|CADH5_HUMAN,sp|P33151|CADH5_HUMAN Cadherin-5 OS=Homo sapie...,P33151,CDH5,0.0
2,MLQIGEDVDYLLIPREVRLAGGVWRVISKPATKEAEFRERLTQFLE...,sp|Q9H0W5|CCDC8_HUMAN,sp|Q9H0W5|CCDC8_HUMAN Coiled-coil domain-conta...,Q9H0W5,CCDC8,candidate
3,MSLPPEKASELKQLIHQQLSKMDVHGRIREILAETIREELAPDQQH...,sp|Q8TAP6|CEP76_HUMAN,sp|Q8TAP6|CEP76_HUMAN Centrosomal protein of 7...,Q8TAP6,CEP76,candidate
4,MARAGPRLVLSEEAVRAKSGLGPHRDLAELQSLSIPGTYQEKITHL...,sp|Q9P209|CEP72_HUMAN,sp|Q9P209|CEP72_HUMAN Centrosomal protein of 7...,Q9P209,CEP72,candidate
...,...,...,...,...,...,...
20410,MALIRKTFYFLFAMFFILVQLPSGCQAGLDFSQPFPSGEFAVCESC...,sp|Q8NG35|D105A_HUMAN,sp|Q8NG35|D105A_HUMAN Beta-defensin 105 OS=Hom...,Q8NG35,DEFB105A,0.0
20411,MRSMKALQKALSRAGSHCGRGGWGHPSRSPLLGGGVRHHLSEAAAQ...,sp|Q9UI32|GLSL_HUMAN,sp|Q9UI32|GLSL_HUMAN Glutaminase liver isoform...,Q9UI32,GLS2,0.0
20412,MNLPRAERLRSTPQRSLRDSDGEDGKIDVLGEEEDEDEEEAASQQF...,sp|Q12950|FOXD4_HUMAN,sp|Q12950|FOXD4_HUMAN Forkhead box protein D4 ...,Q12950,FOXD4,0.0
20413,MCSLPRGFEPQAPEDLAQRSLVELREMLKRQERLLRNEKFICKLPD...,sp|P0CAP2|GRL1A_HUMAN,sp|P0CAP2|GRL1A_HUMAN DNA-directed RNA polymer...,P0CAP2,POLR2M,candidate


# mmseqs2

In [9]:
os.makedirs('mmseqs_clusters', exist_ok=True)
os.makedirs('tmp', exist_ok=True)

In [10]:
# Create Fasta for mmseqs2
f = open(f'seqs_and_labels.fasta', 'w')
for row in seqs_and_labels[['protein_ac', 'seq']].iterrows():
    f.writelines(f'>{row[1][0]}\n{row[1][1]}\n')
f.close()

Run the following command:

!mmseqs easy-cluster seqs_and_labels.fasta mmseqs_clusters/seqs_and_labels_cov_0.1_min_seq_id_0.1_e_0.001 tmp -c 0.1 --min-seq-id 0.1 -e 0.001

For reproducibility, please load the attached cluster file

In [11]:
cluster = pd.read_csv(f'mmseqs_clusters/seqs_and_labels_cov_0.1_min_seq_id_0.1_e_0.001_cluster.tsv', sep='\t', names=['cov_0.1_min_seq_id_0.1_e_0.001_cluster', 'query'])

In [12]:
seqs_and_labels = pd.merge(seqs_and_labels, cluster, 'left', left_on='protein_ac', right_on='query')
seqs_and_labels.head()

,seq,id,description,protein_ac,gene,label,cov_0.1_min_seq_id_0.1_e_0.001_cluster,query
0,MPQLSLSWLGLGPVAASPWLLLLLVGGSWLLARVLAWTYTFYDNCR...,sp|Q9HBI6|CP4FB_HUMAN,sp|Q9HBI6|CP4FB_HUMAN Cytochrome P450 4F11 OS=...,Q9HBI6,CYP4F11,candidate,Q07973,Q9HBI6
1,MQRLMMLLATSGACLGLLAVAAVAAAGANPAQRDTHSLLPTHRRQK...,sp|P33151|CADH5_HUMAN,sp|P33151|CADH5_HUMAN Cadherin-5 OS=Homo sapie...,P33151,CDH5,0.0,Q6V1P9,P33151
2,MLQIGEDVDYLLIPREVRLAGGVWRVISKPATKEAEFRERLTQFLE...,sp|Q9H0W5|CCDC8_HUMAN,sp|Q9H0W5|CCDC8_HUMAN Coiled-coil domain-conta...,Q9H0W5,CCDC8,candidate,A0A1B0GUJ8,Q9H0W5
3,MSLPPEKASELKQLIHQQLSKMDVHGRIREILAETIREELAPDQQH...,sp|Q8TAP6|CEP76_HUMAN,sp|Q8TAP6|CEP76_HUMAN Centrosomal protein of 7...,Q8TAP6,CEP76,candidate,Q9P2K1,Q8TAP6
4,MARAGPRLVLSEEAVRAKSGLGPHRDLAELQSLSIPGTYQEKITHL...,sp|Q9P209|CEP72_HUMAN,sp|Q9P209|CEP72_HUMAN Centrosomal protein of 7...,Q9P209,CEP72,candidate,Q86X45,Q9P209


# Create Data Splits

In [13]:
# Remove candidate samples
training_data = seqs_and_labels[(seqs_and_labels['label'] != 'candidate')].copy()
training_data['label'] = training_data['label'].astype(float)
training_data = training_data.reset_index(drop=True)

In [14]:
# Potentially not reproducing the same split, though tested on another machine and it was the same with our used splits
sgkf = StratifiedGroupKFold(n_splits=6, shuffle=True, random_state=42)
for i, split in enumerate(sgkf.split(training_data, training_data['label'], training_data['cov_0.1_min_seq_id_0.1_e_0.001_cluster'])):
    training_data[f'group_split_{i}'] = 'train'
    training_data.loc[split[1], f'group_split_{i}'] = 'val'
training_data.loc[training_data['group_split_5'] == 'val', ['group_split_0', 'group_split_1', 'group_split_2', 'group_split_3', 'group_split_4']] = 'test'

In [15]:
training_data.to_pickle('training_data.pickle')

#### All without removing uncertain

In [16]:
all_data = seqs_and_labels.merge(training_data[['protein_ac', 'group_split_0', 'group_split_1', 'group_split_2', 'group_split_3', 'group_split_4']], 'left', 'protein_ac')

In [17]:
all_data.to_pickle('all_with_candidates.pickle')